In [1]:
import time
import requests
import pybullet as p
import pybullet_data
import mediapipe as mp

In [2]:
SERVER_URL  = "http://localhost:3000/landmarks"
PLANE_SIZE  = 5.0      # meters
DEPTH_SCALE = 0.5      # scale factor for normalized z
FPS         = 240.0

mp_hands = mp.solutions.hands

In [3]:
def create_spheres(n):
    vis = p.createVisualShape(p.GEOM_SPHERE, radius=0.02, rgbaColor=[1,0,0,1])
    return [p.createMultiBody(baseMass=0, baseVisualShapeIndex=vis, basePosition=[0,0,0])
            for _ in range(n)]

In [4]:
def map_to_plane(x, y, z):
    # Center (0.5,0.5) → (0,0), flip Y so image-up→world +Y
    wx = (x - 0.5) * PLANE_SIZE
    wy = (0.5 - y) * PLANE_SIZE
    wz = -z * DEPTH_SCALE
    return [wx, wy, wz] 

In [6]:
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0, 0, -9.81)
p.setTimeStep(1.0/FPS)

# finite 5×5 plane
p.loadURDF(
    "plane100.urdf",
    basePosition=[0,0,0],
    useFixedBase=True,
    globalScaling=PLANE_SIZE/100.0
)

# spawn spheres
num_points = len(mp_hands.HandLandmark)
spheres = create_spheres(num_points)

try:
    while True:
        # 1) fetch latest landmarks
        try:
            data = requests.get(SERVER_URL, timeout=0.01).json()
        except requests.exceptions.RequestException:
            data = {}

        # 2) update each sphere and collect positions
        world_positions = [None]*num_points
        for name, vals in data.items():
            idx = mp_hands.HandLandmark[name].value
            x, y, z, _ = vals
            pos = map_to_plane(x, y, z)
            world_positions[idx] = pos
            p.resetBasePositionAndOrientation(spheres[idx], pos, [0,0,0,1])

        # 3) draw connections
        for start_idx, end_idx in mp_hands.HAND_CONNECTIONS:
            start = world_positions[start_idx]
            end   = world_positions[end_idx]
            if start and end:
                p.addUserDebugLine(
                    start, end,
                    lineColorRGB=[0,1,0],
                    lineWidth=2,
                    lifeTime=1.0/FPS
                )

        # 4) step sim & throttle
        p.stepSimulation()
        time.sleep(1.0/FPS)

except KeyboardInterrupt:
    pass
finally:
    p.disconnect()

error: Not connected to physics server.